# TabTransformer
---

## Setup & Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import japanize_matplotlib
plt.rcParams["figure.figsize"] = 12, 8

# Module
import os
import sys
import pickle
import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
from torch.autograd import Variable
import pandas as pd
import math
import sklearn.preprocessing as sk
from tensorboardX import SummaryWriter
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split
import random

np.set_printoptions(threshold=np.inf)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

from tqdm import tqdm_notebook as tqdm

from tab_transformer_pytorch import TabTransformer

/Users/fukasawat78/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/japanize_matplotlib/japanize_matplotlib.py:15: MatplotlibDeprecationWarning: 
The createFontList function was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use FontManager.addfont instead.
  font_list = font_manager.createFontList(font_files)


In [2]:
cont_mean_std = torch.randn(10, 2)

model = TabTransformer(
    categories = (10, 10, 10, 10, 10, 10, 10, 10, 10, 10),
    num_continuous = 10,
    dim = 32,
    dim_out = 2,
    depth = 6,
    heads = 10,
    attn_dropout = 0.1,
    ff_dropout = 0.1,
    mlp_hidden_mults = (4, 2),
    mlp_act = nn.ReLU(),
    continuous_mean_std = cont_mean_std
)

X_categ = torch.randint(0, 10, size=(100, 10))
X_cont = torch.randn(100, 10)

X_categ_train = X_categ[:50]
X_cont_train = X_cont[:50]
X_categ_test = X_categ[50:]
X_cont_test = X_categ[50:]

y = torch.randint(0, 2, size=(1, 100))
y_train = y[0, :50]
y_test = y[0, 50:]

In [3]:
def get_optimizer(model, lr = 0.001, wd = 0.0):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optim = torch.optim.Adam(parameters, lr=lr, weight_decay=wd)
    return optim

def train_model(model, optim, X_cat, X_cont, y):
    model.train()
    total = 1
    sum_loss = 0
    for batch, (x1, x2, y1) in enumerate(zip(X_cat, X_cont, y)):
        x1 = x1.unsqueeze(0)
        x2 = x2.unsqueeze(0)
        y1 = y1.unsqueeze(0)
        output = model(x1, x2)
        #prob = F.softmax(out, dim=1)
        #_, pred = torch.max(prob, 1)
        loss = F.cross_entropy(output, y1)   
        optim.zero_grad()
        loss.backward()
        optim.step()
        total += batch
        sum_loss += batch*(loss.item())
        print("loss: {}".format(sum_loss/total))
    return sum_loss/total

optim = get_optimizer(model, lr = 0.01, wd = 0.0)
train_model(model, optim, X_categ_train, X_cont_train, y_train)

loss: 0.0
loss: 0.3465735912322998
loss: 0.457919180393219
loss: 0.2616761751773343
loss: 10.764729543117111
loss: 7.441914598424091
loss: 6.936234243189822
loss: 5.417297808756145
loss: 4.460556559854307
loss: 3.7234547251476253
loss: 3.182328378239942
loss: 2.773656838635035
loss: 2.4576300554192727
loss: 2.2083009538061127
loss: 2.008186304761007
loss: 1.8451649259639318
loss: 1.7106227077450316
loss: 1.5983039809283606
loss: 1.5035782694612227
loss: 1.4229594702311923
loss: 1.3537829500637426
loss: 1.293984022824165
loss: 1.2419430366512894
loss: 1.1963751498415642
loss: 1.1562506607483842
loss: 1.1207365903278486
loss: 1.089153281792495
loss: 1.0609417654815367
loss: 1.035638944045482
loss: 1.012858528710974
loss: 0.9922762532015507
loss: 0.9736183031153425
loss: 0.9566521861761671
loss: 0.9411794724350964
loss: 0.9270299793830882
loss: 0.9140570825651054
loss: 0.9021339095461876
loss: 0.8911502321285473
loss: 0.8810099142212292
loss: 0.8716288046968905
loss: 0.8629329887537825
lo

0.8068455078833755

## Predict

In [4]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_curve, roc_auc_score

model.eval()
out = model(X_categ_test, X_cont_test)
prob = F.softmax(out, dim=1)
_, pred = torch.max(prob, 1)
y_pred = pred.numpy()


print("Confusion matrix:\n {}".format(confusion_matrix(y_test, y_pred)))
print("Classification_report:\n {}".format(classification_report(y_test, y_pred)))
print("Accuracy: {}".format(accuracy_score(y_test, y_pred)))
print("AUC:{}".format(roc_auc_score(y_test, y_pred)))

Confusion matrix:
 [[30  0]
 [20  0]]
Classification_report:
               precision    recall  f1-score   support

           0       0.60      1.00      0.75        30
           1       0.00      0.00      0.00        20

    accuracy                           0.60        50
   macro avg       0.30      0.50      0.37        50
weighted avg       0.36      0.60      0.45        50

Accuracy: 0.6
AUC:0.5


/Users/fukasawat78/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
